In [1]:
import sys, json, time
from threading import Thread, Condition, Event
from psonic import *
sys.path.append("./repo/piplay")
from weather_pilock.SenseHat_Data import Sensor
from pysonic.play_weather import Notes, Rates, rescale


In [23]:
#senseHat.add_stat()
#temp = senseHat.sense.temperature_from_humidity.value
#print(temp)
play(30)
sleep(1)
play(110)
#print(rescale(temp, 40, 110, 50, 80))
#play(rescale(temp, 40, 110, 50, 80))
#senseHat.clear_stats()
#print(beats_per_minute, beats_per_second, collections_per_beat, collection_delay)

#senseHat = Sensor()
#print(senseHat.sense)
#names = list(filter(lambda x: not x.startswith('__') and x., dir(senseHat.sense)))
#print(names)
#senseHat.touch()
#print(senseHat.sense)
#print(senseHat.sense.temperature_from_humidity.value)
#data = senseHat.sense()
#data
#json.dumps(senseHat.reading)



In [2]:
condition = Condition()
stop_event = Event()
rates = Rates()
senseHat = Sensor()

def live_m():
    sleep(rates.seconds_per_beat)
    pass
def live_loop_m(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.notifyAll() #Message to threads
        live_m()

def play_temp():
    pass
def live_loop_temp(condition, stop_event):
     while not stop_event.is_set():
        with condition:
            condition.wait() #Message to threads
        play_temp()

def play_pressure():
    pass    
def live_loop_pressure(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_pressure()

def play_humidity():
    pass
def live_loop_humidity(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_humidity()


live_thread_m = Thread(name="producer", target=live_loop_m, args=(condition, stop_event))
live_thread_t = Thread(name="consumer_temp", target=live_loop_temp, args=(condition, stop_event))
live_thread_1 = Thread(name="consumer_pressure", target=live_loop_pressure, args=(condition, stop_event))
live_thread_2 = Thread(name="consumer_humidity", target=live_loop_humidity, args=(condition, stop_event))

live_thread_m.start()
live_thread_t.start()
live_thread_1.start()
live_thread_2.start()



In [3]:
def play_weather(senseHat):
    while True:
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.temperature_from_humidity), 
                 rescale(senseHat.sense.temperature_from_pressure)], 
                     attack = 0.5, 
                     amp = 0.5
                    )
        pressure = Notes([rescale(senseHat.sense.pressure) / 2.0],
                         attack = 0.5, 
                         amp = 0.5
                        )
        humidity = Notes([rescale(senseHat.sense.humidity) / 3.0],
                         attack = 0.5, 
                         amp = 0.5
                        )
        
        global play_temp
        def play_temp():
            play(temp.notes, attack=temp.attack, decay=temp.decay, sustain=temp.sustain, release=temp.release, amp=temp.amp)
            pass
    
        global play_pressure
        def play_pressure():
            play(pressure.notes, attack=pressure.attack, decay=pressure.decay, sustain=pressure.sustain, release=pressure.release, amp=pressure.amp)
            pass
    
        global play_humidity
        def play_humidity():
            play(humidity.notes, attack=humidity.attack, decay=humidity.decay, sustain=humidity.sustain, release=humidity.release, amp=humidity.amp)
            pass

In [4]:
play_weather(senseHat)

KeyboardInterrupt: 

In [5]:
stop_event.set()